In [117]:
workflows, parts = open("./data/sample_day19.txt").read().split("\n\n")
workflows = workflows.split("\n")
parts = parts.split("\n")
workflows, parts

(['px{a<2006:qkq,m>2090:A,rfg}',
  'pv{a>1716:R,A}',
  'lnx{m>1548:A,A}',
  'rfg{s<537:gd,x>2440:R,A}',
  'qs{s>3448:A,lnx}',
  'qkq{x<1416:A,crn}',
  'crn{x>2662:A,R}',
  'in{s<1351:px,qqz}',
  'qqz{s>2770:qs,m<1801:hdj,R}',
  'gd{a>3333:R,R}',
  'hdj{m>838:A,pv}'],
 ['{x=787,m=2655,a=1222,s=2876}',
  '{x=1679,m=44,a=2067,s=496}',
  '{x=2036,m=264,a=79,s=2244}',
  '{x=2461,m=1339,a=466,s=291}',
  '{x=2127,m=1623,a=2188,s=1013}'])

In [118]:
import re
from pprint import pprint

parsed = {}
for workflow in workflows:
    rule = f"[xmas][<>]\d+:\w+,?"
    w = re.match(r"""(?P<id>\w+)
                    \{(?P<r1>[xmas][<>]\d+:\w+,?)
                    (?P<r2>[xmas][<>]\d+:\w+,?)?
                    (?P<r3>[xmas][<>]\d+:\w+,?)?
                    (?P<r4>[xmas][<>]\d+:\w+,?)?
                    (?P<default>\w+)\}""", workflow, flags=re.VERBOSE)
    w = w.groupdict()
    parsed[w["id"]] = []
    for k, v in w.items():
        if k.startswith("r") and v is not None:
            rule = re.match(r"(?P<part>[xmas])(?P<cond>[<>])(?P<value>\d+):(?P<next>\w+),?", v).groupdict()
            pt = rule["part"]
            
            parsed[w["id"]].append((pt, {"cond": rule["cond"], "value": int(rule["value"]), "next": rule["next"]}))
    parsed[w["id"]].append(("default", {"next": w["default"]}))
pprint(parsed)
            


{'crn': [('x', {'cond': '>', 'next': 'A', 'value': 2662}),
         ('default', {'next': 'R'})],
 'gd': [('a', {'cond': '>', 'next': 'R', 'value': 3333}),
        ('default', {'next': 'R'})],
 'hdj': [('m', {'cond': '>', 'next': 'A', 'value': 838}),
         ('default', {'next': 'pv'})],
 'in': [('s', {'cond': '<', 'next': 'px', 'value': 1351}),
        ('default', {'next': 'qqz'})],
 'lnx': [('m', {'cond': '>', 'next': 'A', 'value': 1548}),
         ('default', {'next': 'A'})],
 'pv': [('a', {'cond': '>', 'next': 'R', 'value': 1716}),
        ('default', {'next': 'A'})],
 'px': [('a', {'cond': '<', 'next': 'qkq', 'value': 2006}),
        ('m', {'cond': '>', 'next': 'A', 'value': 2090}),
        ('default', {'next': 'rfg'})],
 'qkq': [('x', {'cond': '<', 'next': 'A', 'value': 1416}),
         ('default', {'next': 'crn'})],
 'qqz': [('s', {'cond': '>', 'next': 'qs', 'value': 2770}),
         ('m', {'cond': '<', 'next': 'hdj', 'value': 1801}),
         ('default', {'next': 'R'})],
 'qs':

In [119]:
parts = [p[1:-1].split(",") for p in parts]
parts[:2]

[['x=787', 'm=2655', 'a=1222', 's=2876'],
 ['x=1679', 'm=44', 'a=2067', 's=496']]

In [120]:
parts = [dict([(c.split("=")[0], int(c.split("=")[1])) for c in p]) for p in parts]
parts[:2]

[{'x': 787, 'm': 2655, 'a': 1222, 's': 2876},
 {'x': 1679, 'm': 44, 'a': 2067, 's': 496}]

In [121]:
import operator

op = {">": operator.gt, "<": operator.lt}


def run_workflow(wf, part):
    nwf = None
    rule = parsed[wf]
    for c, r in rule:
        if c == "default":
            nwf = r["next"]
            break
        if op[r["cond"]](part[c], r["value"]):
            nwf = r["next"]
            break
    return nwf

run_workflow('in', {'x': 903, 'm': 143, 'a': 1348, 's': 25})

'px'

In [122]:
total = 0
accepted = []
rejected = []
for part in parts:
    wp = "in"
    while wp not in "RA":
        print(wp, "->", end=" ")
        wp = run_workflow(wp, part)
    print(wp)
    if wp == "A":
        accepted.append(part)
        total += sum(part.values())
    else:
        rejected.append(part)

total

in -> qqz -> qs -> lnx -> A
in -> px -> rfg -> gd -> R
in -> qqz -> hdj -> pv -> A
in -> px -> qkq -> crn -> R
in -> px -> rfg -> A


19114